In [ ]:
from google.colab import files
files.upload()

In [ ]:
!pip install kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d ashishjangra27/face-mask-12k-images-dataset

In [ ]:
!unzip face-mask-12k-images-dataset.zip -d data

In [ ]:

import json
import math
import os

import cv2
from PIL import Image
import numpy as np
from keras import layers
from keras.applications import ResNet50,MobileNet
from keras.callbacks import Callback, ModelCheckpoint, ReduceLROnPlateau, TensorBoard
from keras.preprocessing.image import ImageDataGenerator
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.optimizers import Adam
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score, accuracy_score
import scipy
from tqdm import tqdm
import tensorflow as tf
from keras import backend as K
import gc
from functools import partial
from sklearn import metrics
from collections import Counter
import itertools


%matplotlib inline

In [ ]:

#Transfer 'png' images to an array IMG
def Dataset_loader(DIR, RESIZE, sigmaX=10):
    IMG = []
    read = lambda imname: np.asarray(Image.open(imname).convert("RGB"))
    for IMAGE_NAME in tqdm(os.listdir(DIR)):
        PATH = os.path.join(DIR,IMAGE_NAME)
        _, ftype = os.path.splitext(PATH)
        if ftype == ".png":
            img = read(PATH)

            img = cv2.resize(img, (RESIZE,RESIZE))

            IMG.append(np.array(img))
    return IMG

train_data_withoutmask = np.array(Dataset_loader('/content/data/Face Mask Dataset/Train/WithoutMask',150))
tran_data_withmask = np.array(Dataset_loader('/content/data/Face Mask Dataset/Train/WithMask',150))
test_data_withoutmask = np.array(Dataset_loader('/content/data/Face Mask Dataset/Test/WithoutMask',150))
test_data_withmask = np.array(Dataset_loader('/content/data/Face Mask Dataset/Test/WithMask',150))

In [ ]:
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import load_img
import cv2
import random


# Input data files are available in the read-only "../input/" directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
os.listdir("/content/data/Face Mask Dataset/Train")

In [ ]:
print("Images in Train Dataset:\n")
print("Number of Images for with Mask category:{}".format(len(os.listdir("/content/data/Face Mask Dataset/Train/WithMask"))))
print("Number of Images for with WithoutMask category:{}".format(len(os.listdir("/content/data/Face Mask Dataset/Train/WithoutMask"))))


In [ ]:
train_dir = "/content/data/Face Mask Dataset/Train/"
test_dir = "/content/data/Face Mask Dataset/Test"
validation_dir = "/content/data/Face Mask Dataset/Validation"

In [ ]:
#with Mask
plt.figure(figsize=(12,7))
for i in range(5):
    sample = random.choice(os.listdir(train_dir+"WithMask/"))
    plt.subplot(1,5,i+1)
    img = load_img(train_dir+"WithMask/"+sample)
    plt.subplots_adjust(hspace=0.001)
    plt.xlabel("With Mask")
    plt.imshow(img)
plt.show()

#without Mask
plt.figure(figsize=(12,7))
for i in range(5):
    sample = random.choice(os.listdir(train_dir+"WithoutMask/"))
    plt.subplot(1,5,i+1)
    img = load_img(train_dir+"WithoutMask/"+sample)
    plt.subplots_adjust(hspace=0.001)
    plt.xlabel("Without Mask")
    plt.imshow(img)
plt.show()

In [ ]:
#Image agmentation

height = 150
width=150
train_datagen = ImageDataGenerator(rescale=1.0/255,validation_split=0.2,shear_range = 0.2,zoom_range=0.2,horizontal_flip=True)

train = train_datagen.flow_from_directory(directory=train_dir,target_size=(height,width),
                                          class_mode="categorical",batch_size=32,subset = "training")

valid_datagen = ImageDataGenerator(rescale=1.0/255)

valid = train_datagen.flow_from_directory(directory=train_dir,target_size=(height,width),
                                          class_mode="categorical",batch_size=32,subset="validation")

In [ ]:
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from keras.applications import ResNet50,MobileNet, DenseNet201, InceptionV3, NASNetLarge, InceptionResNetV2, NASNetMobile
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications import vgg16

resnet = ResNet50(include_top=False,  weights='imagenet', input_shape=(150,150,3))

In [ ]:
for layer in vgg.layers:
    layer.trainable = False

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten,Dense
model = Sequential()
model.add(resnet)
model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dropout(0.5))
model.add(layers.BatchNormalization())
model.add(layers.Dense(2, activation='softmax'))

In [ ]:
from keras import backend as K
def sensitivity(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    return true_positives / (possible_positives + K.epsilon())

def specificity(y_true, y_pred):
    true_negatives = K.sum(K.round(K.clip((1-y_true) * (1-y_pred), 0, 1)))
    possible_negatives = K.sum(K.round(K.clip(1-y_true, 0, 1)))
    return true_negatives / (possible_negatives + K.epsilon())

In [ ]:
model.compile(optimizer="adam",
              loss="categorical_crossentropy",
              metrics=['accuracy',sensitivity, specificity])


from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping

checkpoint = ModelCheckpoint("resnet_facemask.h5",
                             monitor="val_accuracy",
                             save_best_only=True,verbose=1)

earlystop = EarlyStopping(monitor="val_acc",patience=5,verbose=1)

In [ ]:
history = model.fit_generator(generator=train,steps_per_epoch=len(train)// 32,
                              validation_data=valid,
                             validation_steps = len(valid)//32,
                              callbacks=[checkpoint,earlystop],
                              epochs=20)

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
loss = history.history['loss']
val_loss = history.history['val_loss']
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
loss = history.history['loss']
val_loss = history.history['val_loss']
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

In [ ]:
import keras
from matplotlib import pyplot as plt
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
sensitivity = history.history['sensitivity']
val_sensitivity = history.history['val_sensitivity']
specificity = history.history['specificity']
val_specificity = history.history['val_specificity']
epochs = range(1, len(sensitivity) + 1)
plt.plot(epochs, sensitivity, 'g', label='Training sensitivity')
plt.plot(epochs, val_sensitivity, 'y', label='Validation sensitivity')
plt.title('Training and validation sensitivity')
plt.legend()
plt.figure()
plt.plot(epochs, specificity, 'g', label='Training specificity')
plt.plot(epochs, val_specificity, 'y', label='Validation specificity')
plt.title('Training and validation specificity')
plt.legend()
plt.show


